# Benchmarking Performance of NetworkX with Rapids GPU-based nx_cugraph backend vs on cpu
# Skip notebook test
This notebook demonstrates compares the performance of nx_cugraph as a dispatcher for NetworkX algorithms. 

We do this by executing Betweenness Centrality, Breadth First Search and Louvain Community Detection, collecting run times with and without nx_cugraph backend and graph caching enabled. nx_cugraph is a registered NetworkX backend. Using it is a zero code change solution.

In the notebook switching to the nx-cugraph backend is done via variables set using the [NetworkX config package](https://networkx.org/documentation/stable/reference/backends.html#networkx.utils.configs.NetworkXConfig) **which requires networkX 3.3 or later !!**


They can be set at the command line as well.

### See this example from GTC Spring 2024



Here is a sample minimal script to demonstrate No-code-change GPU acceleration using nx-cugraph.

----
bc_demo.ipy:

```
import pandas as pd
import networkx as nx

url = "https://data.rapids.ai/cugraph/datasets/cit-Patents.csv"
df = pd.read_csv(url, sep=" ", names=["src", "dst"], dtype="int32")
G = nx.from_pandas_edgelist(df, source="src", target="dst")

%time result = nx.betweenness_centrality(G, k=10)
```
----
Running it with the nx-cugraph backend looks like this:
```
user@machine:/# ipython bc_demo.ipy
CPU times: user 7min 38s, sys: 5.6 s, total: 7min 44s
Wall time: 7min 44s

user@machine:/# NETWORKX_BACKEND_PRIORITY=cugraph ipython bc_demo.ipy
CPU times: user 18.4 s, sys: 1.44 s, total: 19.9 s
Wall time: 20 s
```
----


First import the needed packages

In [ ]:
import pandas as pd
import networkx as nx
import time
import os

This installs the NetworkX cuGraph dispatcher if not already present.

In [ ]:
try: 
    import nx_cugraph
except ModuleNotFoundError:
    os.system('conda install -c rapidsai -c conda-forge -c nvidia nx-cugraph')

This is boiler plate NetworkX code to run:
* betweenness Centrality
* Bredth first Search
* Louvain community detection

and report times. it is completely unaware of cugraph or GPU-based tools.
[NetworkX configurations](https://networkx.org/documentation/stable/reference/utils.html#backends) can determine how they are run.

In [ ]:
def run_algos(G):
    runtime = time.time()
    result = nx.betweenness_centrality(G, k=10)
    print ("Betweenness Centrality time: " + str(round(time.time() - runtime))+ " seconds")
    runtime = time.time()
    result = nx.bfs_tree(G,source=1)
    print ("Breadth First Search time:  " + str(round(time.time() - runtime))+ " seconds")
    runtime = time.time()
    result = nx.community.louvain_communities(G,threshold=1e-04)
    print ("Louvain time: " + str(round(time.time() - runtime))+ " seconds")
    return

Downloads a patent citation dataset containing 3774768 nodes and 16518948 edges and loads it into a NetworkX graph.

In [ ]:
filepath = "./data/cit-Patents.csv"

if os.path.exists(filepath):
    print("File found")
    url = filepath
else:
    url = "https://data.rapids.ai/cugraph/datasets/cit-Patents.csv"
df = pd.read_csv(url, sep=" ", names=["src", "dst"], dtype="int32")
G = nx.from_pandas_edgelist(df, source="src", target="dst")

Setting the NetworkX dispatcher with an environment variable or in code using NetworkX config package which is new to [NetworkX 3.3 config](https://networkx.org/documentation/stable/reference/backends.html#networkx.utils.configs.NetworkXConfig).

These convenience settinge allow turning off caching and cugraph dispatching if you want to see how long cpu-only takes.
This example using an AMD Ryzen Threadripper PRO 3975WX 32-Cores cpu completed in slightly over 40 minutes.

In [ ]:
use_cugraph = True
cache_graph = True

In [ ]:
if use_cugraph:
    nx.config["backend_priority"]=['cugraph']
else:
    # Use this setting to turn off the cugraph dispatcher running in legacy cpu mode.
    nx.config["backend_priority"]=[]
if cache_graph:
    nx.config["cache_converted_graphs"]= True
else:
    # Use this setting to turn off graph caching which will convertthe NetworkX to a gpu-resident graph each time an algorithm is run.
    nx.config["cache_converted_graphs"]= False


Run the algorithms on GPU. 

**Note the messages NetworkX generates to remind us cached graph shouldn't be modified.**

```
For the cache to be consistent (i.e., correct), the input graph must not have been manually mutated since the cached graph was created.

Using cached graph for 'cugraph' backend in call to bfs_edges.
```

In [ ]:
%%time
run_algos(G)
print ("Total Algorithm run time")

___
Copyright (c) 2024, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___